### Inicializando e checando o ecossistema de databases

In [10]:
#mongodb
!mongo --quiet --eval 'rs.initiate();'
!mongo --quiet --eval 'rs.isMaster().ismaster'

#redis
!redis-server --daemonize yes
!redis-cli ping

#cassandra
!cqlsh --execute="SELECT host_id, bootstrapped FROM system.local;"
!cqlsh --file=./cassandra/initialize.cql


 host_id                              | bootstrapped
--------------------------------------+--------------
 d9c74d7a-cef4-43ba-99ec-0b8f2b07c85c |    COMPLETED

(1 rows)


### Inicializando modulos, libs e variaveis

In [11]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
import json
from cassandra.cluster import Cluster
from libs.dispute_faker import generate_dispute


warnings.filterwarnings('ignore')

mongoclient = MongoClient('localhost', 27017)
dispute_db = mongoclient["disputes"]
print(dispute_db)

cluster = Cluster()
cassandra = cluster.connect('disputes')


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'disputes')


In [12]:
inserted = dispute_db['disputes'].insert_many([generate_dispute() for _ in range(2000)])
pprint(len(inserted.inserted_ids))

2000


In [13]:
random_dispute = dispute_db['disputes'].find().limit(1)[0]
random_dispute['id'] = str(random_dispute['_id'])
random_dispute['gmtCreated'] = str(random_dispute['gmtCreated'])
random_dispute['gmtModified'] = str(random_dispute['gmtModified'])
random_dispute.pop('_id')
random_dispute = json.dumps(random_dispute)
pprint(random_dispute)
pprint(cassandra.execute(f"INSERT INTO disputes JSON'{random_dispute}'")[0])

'{"disputeReason": "WRONG_ITEM", "buyerId": "56", "gmtCreated": "1672950907", "gmtModified": "1673171573", "seller": {"storeId": "43", "sellerId": "25"}, "item": {"itemPrice": {"cent": 49164370, "currency": "SRD"}, "supportFreeReturn": false, "itemCount": 45, "itemId": "69", "skuId": "72", "skuValue": "eTA", "tradeSnapshotUrl": "https://jailed-market.com/product/69/details?snapshot=2023-07-13", "ParentOrderId": "86"}, "resolution": {"status": "WAITING_FOR_BUYER", "arbitratorId": null, "gmtArbitratorAssigned": null, "gmtArbitrated": null, "decision": null, "first_round": {"initialProposal": {"gmtSubmitted": 1672956445, "richTextReport": "After space have message itself hand mean factor add more live.", "proposedSolution": "REPLACE_ITEM", "status": "COUNTERED", "attachments": []}, "counterProposal": {"gmtSubmitted": 1673171573, "richTextReport": "Place business back himself force somebody dog color Republican trouble drive back small girl bank out.", "proposedSolution": "SHIP_MISSING_QUA

InvalidRequest: Error from server: code=2200 [Invalid query] message="Error decoding JSON value for resolution: Unknown field 'first_round' in value of user defined type resolution"

In [ ]:
pprint(cassandra.execute(f"DROP keyspace disputes"))